Needed imports and utility

In [ ]:
import os
import ansys.speos.core as core

def clean_all_dbs(speos_client: core.SpeosClient):
    for item in (
        speos_client.jobs().list()
        + speos_client.scenes().list()
        + speos_client.simulation_templates().list()
        + speos_client.sensor_templates().list()
        + speos_client.source_templates().list()
        + speos_client.intensity_templates().list()
        + speos_client.spectrums().list()
        + speos_client.vop_templates().list()
        + speos_client.sop_templates().list()
        + speos_client.parts().list()
        + speos_client.bodies().list()
        + speos_client.faces().list()
    ):
        item.delete()

# Create connection with speos rpc server

In [ ]:
speos = core.Speos(host="localhost", port=50098)
clean_all_dbs(speos.client)

# Create an ObjectLink

In [ ]:
sop_t_db = speos.client.sop_templates()

In [ ]:
sop_t = core.SOPTemplate()
sop_t.name = "Mirror_90"
sop_t.mirror.reflectance = 90.0

mirror_90_link = sop_t_db.create(message=sop_t)
print(mirror_90_link)

In [ ]:
sop_t = core.SOPTemplate()
sop_t.name = "Mirror_100"
sop_t.mirror.reflectance = 100.0
mirror_100_link = sop_t_db.create(message=sop_t)

# Modify an ObjectLink

In [ ]:
# Modify data locally
mirror_data = mirror_90_link.get()
mirror_data.name = "Mirror_50"
mirror_data.mirror.reflectance = 50

# Update on db
mirror_90_link.set(data=mirror_data)
print(mirror_90_link)


# List ObjectLinks

In [ ]:
sop_t_links = sop_t_db.list()
for s in sop_t_links:
    print(s)

# Delete an ObjectLink

This means deleting data in db

In [ ]:
mirror_100_link.delete()
mirror_90_link.delete()
assert len(sop_t_db.list()) == 0

# Find from key

In [ ]:
tests_data_path = os.path.join(os.path.join(os.path.abspath(''), os.path.pardir), "assets")
speos_file = os.path.join(tests_data_path, os.path.join("LG_50M_Colorimetric_short.sv5", "LG_50M_Colorimetric_short.sv5"))

scene_link = speos.client.scenes().create()
scene_link.load_file(file_uri=speos_file)
print(scene_link)

In [ ]:
scene_data = scene_link.get()
assert len(scene_data.sensors) > 0
sensor_t_link = speos.client.get_item(key=scene_data.sensors[0].sensor_guid)
assert isinstance(sensor_t_link, core.SensorTemplateLink)
print(sensor_t_link)

# Clean all dbs

In [ ]:
clean_all_dbs(speos.client)